#### Introduction et objectifs du projet

##### <u>Contexte :</u>

L'analyse de sentiments est un domaine essentiel en science des données, en particulier en NLP. Elle utilise le machine learning et la linguistique pour détecter, extraire et étudier les émotions et les informations subjectives dans les textes sources.

Dans le contexte des avis de films, l'analyse de sentiments permet de comprendre les perceptions et les émotions des spectateurs à partir de leurs commentaires, ce qui est précieux pour les critiques de films, les studios de cinéma et les plateformes de streaming.

##### <u>Objectifs :</u>

Ce projet vise à développer une application interactive en Python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques. Les objectifs de l'application sont les suivants :

1. Permettre aux utilisateurs de saisir un titre de film ou son ID pour récupérer des avis pertinents.

2. Analyser les sentiments exprimés dans ces avis, en calculant la polarité (positivité ou négativité des sentiments) et la subjectivité (mesure de l'opinion personnelle par rapport à un fait objectif) des commentaires.

3. Comparer les "ratings" (notes) des films avec la polarité des avis pour voir si les évaluations numériques correspondent aux sentiments exprimés dans les commentaires.

4. Examiner les différences, le cas échéant, entre les sentiments exprimés avant et après la sortie du film, fournissant ainsi une perspective unique sur l'évolution des perceptions des spectateurs.

##### <u>Méthodes de récupération des données :</u>

Dans un premier temps, pour éviter de surcharger l'API de TMDB et de devoir gérer des bases de données trop volumineuses, nous avons décidé de limiter notre étude aux films sortis entre x et y. Une fois cette sélection effectuée, nous avons procédé à la récupération des données via 3 méthodes différentes :

- L'API de TMDB : "https://developers.themoviedb.org/3/"
- Scraping du site : imdb.com (via la librairie python : beautifulsoup4)
- Utilisation de la librairie python : imdbpy

L'objectif de cette section est de comparer les différentes méthodes de récupération des données afin de déterminer laquelle est la plus efficace et la plus pertinente pour notre projet.
Dans une première approche il était question de tester les 3 méthodes, voir les avantages et inconvénients de chacune d'entre elles, de savoir si elles étaient complémentaires ou non, et de déterminer laquelle était la plus efficace et la plus pertinente pour notre projet.

Dans un second temps l'objectif est de structurer les données récupérées afin de pouvoir les exploiter dans la suite du projet. Si possible par une seule des méthodes de récupération des données mentionnées ci-dessus.

Ensuite, il s'agira de mettre en place une méthode d'analyse de sentiments sur les commentaires récupérés, il existe des librairies python qui permettent de faire cela, mais il est également possible de créer notre propre méthode d'analyse de sentiments. 

Choix numéro 1 : Réaliser nos propres méthodes d'analyse de sentiments.

Choix numéro 2 : Utiliser des librairies python existantes tel que : TextBlob, NLTK.

Enfin il s'agira de mettre en place une application interactive en python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques.


#### <u>Import des librairies</u>

- package imdb à importer via le code ci-dessous : (important qu'il soit installé sur la machine et qu'il soit à jour)

In [1]:
#pip install git+https://github.com/santhoshse7en/imdb

In [2]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import requests
from datetime import datetime
import json
import re
import pandas as pd

- Appel de la clé API de TMDB 

In [3]:
with open('clefAPI.json', 'r') as config_file:
    config = json.load(config_file)

api_key = config['tmdb']['api_key']

#### <u>Méthode 1 : scraping du site imdb.com </u>

Première étape : on va tester notre procédé sur un seul film, pour voir si ça fonctionne, et si oui, on pourra ensuite l'appliquer à tous les films de notre base de données.

Étape crucial : on doit récupérer la date de sortie du film, pour pouvoir ensuite récupérer les commentaires avant et après la sortie du film. 

The Shawshank Redemption : ID sur le site de TMDB : "tt0111161"

In [4]:
film_id = 'tt0111161'

url_date_release = f"https://api.themoviedb.org/3/movie/{film_id}?api_key={api_key}&language=en-US"

response = requests.get(url_date_release)

if response.status_code == 200:
    film_details = response.json()
    date_sortie_str = film_details.get("release_date", "")
    if date_sortie_str:
        date_sortie = datetime.strptime(date_sortie_str, "%Y-%m-%d")
        print(f"Date de sortie du film : {date_sortie}")
    else:
        print("Date de sortie non trouvée")
else:
    print("Erreur lors de la requête :", response.status_code)

url_film = f"https://www.imdb.com/title/{film_id}/reviews" # Récupérer les avis sur IMDb

response = requests.get(url_film)

reviews_dict = {}

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    review_elements = soup.find_all('div', class_='review-container')
    for element in review_elements:
        review_text = element.find('div', class_='text show-more__control').get_text().strip()
        review_date_str = element.find('span', class_='review-date').get_text().strip()
        review_date = datetime.strptime(review_date_str, '%d %B %Y').date()
        reviews_dict[review_date] = review_text
else:
    print("Erreur lors de la requête :", response.status_code)

print("Nombre d'avis pour ce film :",len(reviews_dict)) 

Date de sortie du film : 1994-09-23 00:00:00
Nombre d'avis pour ce film : 25


- Objectif : voir si on a assez de commentaires avant et après la sortie du film pour pouvoir faire une analyse de sentiments.

In [5]:
avis_avant = {}
avis_apres = {}

for review_date, review_text in reviews_dict.items():
    if review_date < date_sortie.date():
        avis_avant[review_date] = review_text
    else:
        avis_apres[review_date] = review_text

print("Nombre d'avis avant la sortie du film :", len(avis_avant))
print("Nombre d'avis après la sortie du film :", len(avis_apres))

Nombre d'avis avant la sortie du film : 0
Nombre d'avis après la sortie du film : 25


On constate alors qu'avant la sortie du film, on a 0 commentaires, et après la sortie du film, on a 25 commentaire. On ne peut donc pas faire d'analyse de sentiments sur ce film. Et en testant sur d'autres films, on se rend compte que c'est le cas pour tous des films de notre base de données. Afin de réaliser le projet dans son intégralité, il faudrait donc récupérer les commentaires sur une autre plateforme que TMDB. Toutefois nous manquons de temps et d'expérience pour réaliser cela. Nous allons donc nous contenter de réaliser une analyse de sentiments sur les commentaires récupérés sur TMDB.

- Affichage des 5 premiers commentaires récupérés sur TMDB pour ce premier film :

In [6]:
print("Avis après la sortie:")
for date, review in list(avis_apres.items())[:5]:
    print(f"{date}: {review}\n")

Avis après la sortie:
2010-07-24: The Shawshank Redemption is written and directed by Frank Darabont. It is an adaptation of the Stephen King novella Rita Hayworth and Shawshank Redemption. Starring Tim Robbins and Morgan Freeman, the film portrays the story of Andy Dufresne (Robbins), a banker who is sentenced to two life sentences at Shawshank State Prison for apparently murdering his wife and her lover. Andy finds it tough going but finds solace in the friendship he forms with fellow inmate Ellis "Red" Redding (Freeman). While things start to pick up when the warden finds Andy a prison job more befitting his talents as a banker. However, the arrival of another inmate is going to vastly change things for all of them.There was no fanfare or bunting put out for the release of the film back in 94, with a title that didn't give much inkling to anyone about what it was about, and with Columbia Pictures unsure how to market it, Shawshank Redemption barely registered at the box office. Howe

#### <u>Méthode 2 : utilisation de l'API </u>

In [41]:
def get_movies_between_dates(start_date, end_date, api_key):
    base_url = "https://api.themoviedb.org/3/discover/movie"
    url = f"{base_url}?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&region=FR"
    response = requests.get(url)
    if response.status_code == 200:
        movies_data = response.json()['results']
        return [(movie['id'], movie['title']) for movie in movies_data]
    else:
        print("Erreur lors de la requête :", response.status_code)
        return []
    
start_date = "2022-01-01" # FONCTIONNE ! On peut donc définir la période de recherche que l'on souhaite
end_date = "2022-01-31" 

print(f"Recherche des films sortis entre le {start_date} et le {end_date}", end=" ")
print(get_movies_between_dates(start_date, end_date, api_key))

Recherche des films sortis entre le 2022-01-01 et le 2022-12-31 [(76600, 'Avatar: The Way of Water'), (457232, 'Lamborghini: The Man Behind the Legend'), (315162, 'Puss in Boots: The Last Wish'), (606870, 'The Survivor'), (436270, 'Black Adam'), (833326, 'Zero Fucks Given'), (414906, 'The Batman'), (505642, 'Black Panther: Wakanda Forever'), (675353, 'Sonic the Hedgehog 2'), (361743, 'Top Gun: Maverick'), (1028318, 'Goodbye Monster'), (585083, 'Hotel Transylvania: Transformania'), (453395, 'Doctor Strange in the Multiverse of Madness'), (507086, 'Jurassic World Dominion'), (438148, 'Minions: The Rise of Gru'), (829560, 'The Next 365 Days'), (616037, 'Thor: Love and Thunder'), (760104, 'X'), (899112, 'Violent Night'), (744276, 'After Ever Happy')]


In [47]:
def get_movies_between_dates(start_date, end_date, api_key):
    base_url = "https://api.themoviedb.org/3/discover/movie"
    movies = []
    page = 1
    total_pages = 1

    while page <= total_pages:
        url = f"{base_url}?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&region=FR&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            total_pages = data['total_pages']
            movies.extend([(movie['id'], movie['title']) for movie in data['results']])
            page += 1
        else:
            print("Erreur lors de la requête :", response.status_code)
            break
    return movies

def get_movie_reviews(movie_id, api_key):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['results']
    else:
        print(f"Erreur lors de la récupération des avis pour {movie_id}: {response.status_code}")
        return None

def filter_movies_with_reviews(movies, api_key):
    movies_with_reviews = []
    for movie_id, movie_title in movies:
        reviews = get_movie_reviews(movie_id, api_key)
        if reviews and len(reviews) > 0:
            movies_with_reviews.append((movie_id, movie_title))
    return movies_with_reviews


start_date = "2022-01-01"
end_date = "2022-12-31"

print(f"Recherche des films sortis entre le {start_date} et le {end_date}")

movies = get_movies_between_dates(start_date, end_date, api_key)
movies_with_reviews = filter_movies_with_reviews(movies, api_key)

print(f"Nombre de films avec des avis en janvier 2022 : {len(movies_with_reviews)}")

print(f"Nombre de films trouvés : {len(movies)}")

Recherche des films sortis entre le 2022-01-01 et le 2022-01-31
Nombre de films avec des avis en janvier 2022 : 15
Nombre de films trouvés : 266


#### <u>Méthode 3 : utilisation de la librairie python : imdbpy</u>

In [8]:
# from cinemagoer import IMDb
# ia = IMDb()
# movie = ia.get_movie('0111161')  # Pour 'The Shawshank Redemption'
# print(movie['title'])  # Afficher le titre du film
!pip show IMDbPY

Name: IMDbPY
Version: 2022.7.9
Summary: IMDbPY is now cinemagoer
Home-page: https://cinemagoer.github.io/
Author: 
Author-email: 
License: 
Location: C:\Users\pc\anaconda3\Lib\site-packages
Requires: cinemagoer
Required-by: 


#### <u>Récupération des données via la méthode de scraping du site imdb.com</u>

In [9]:
def get_imdb_reviews(movie_id): # FONCTIONNE
    url = f"https://www.imdb.com/title/{movie_id}/reviews"
    response = requests.get(url)
    reviews = {}
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', class_='review-container')
        if not review_elements:
            print(f"Aucune critique trouvée pour {movie_id}")
            return reviews
        for element in review_elements:
            review_text = element.find('div', class_='text show-more__control').get_text().strip()
            review_date = datetime.strptime(element.find('span', class_='review-date').get_text(), '%d %B %Y')
            reviews[review_date] = review_text
    else:
        print(f"Erreur lors de la récupération des avis pour {movie_id}: {response.status_code}")
    return reviews

print(get_imdb_reviews('tt0111161'))

{datetime.datetime(2010, 7, 24, 0, 0): 'The Shawshank Redemption is written and directed by Frank Darabont. It is an adaptation of the Stephen King novella Rita Hayworth and Shawshank Redemption. Starring Tim Robbins and Morgan Freeman, the film portrays the story of Andy Dufresne (Robbins), a banker who is sentenced to two life sentences at Shawshank State Prison for apparently murdering his wife and her lover. Andy finds it tough going but finds solace in the friendship he forms with fellow inmate Ellis "Red" Redding (Freeman). While things start to pick up when the warden finds Andy a prison job more befitting his talents as a banker. However, the arrival of another inmate is going to vastly change things for all of them.There was no fanfare or bunting put out for the release of the film back in 94, with a title that didn\'t give much inkling to anyone about what it was about, and with Columbia Pictures unsure how to market it, Shawshank Redemption barely registered at the box offic

#### <u>Récupération des données via l'API de TMDB</u>

In [11]:
def get_movie_details(movie_id, api_key):
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Erreur lors de la requête :", response.status_code)
        return None

print(get_movie_details('tt0111161', api_key))

{'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'belongs_to_collection': None, 'budget': 25000000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}], 'homepage': '', 'id': 278, 'imdb_id': 'tt0111161', 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 110.352, 'poster_path': '/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg', 'production_companies': [{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxYqAlPPykp.png', 'name': 'Castle Rock Entertainment', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'Unit

In [12]:
def get_movie_details_and_reviews(movie_id, api_key):
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    movie_details = response.json() if response.status_code == 200 else {}
    reviews_url = f"{base_url}{movie_id}/reviews?api_key={api_key}"
    reviews_response = requests.get(reviews_url)
    reviews_data = reviews_response.json() if reviews_response.status_code == 200 else {}

    return movie_details, reviews_data


print(get_movie_details_and_reviews('tt0111161', api_key))

({'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'belongs_to_collection': None, 'budget': 25000000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}], 'homepage': '', 'id': 278, 'imdb_id': 'tt0111161', 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 110.352, 'poster_path': '/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg', 'production_companies': [{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxYqAlPPykp.png', 'name': 'Castle Rock Entertainment', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'Uni

#### <u>On transforme nos données en json</u>

In [45]:
movies_data = get_movies_between_dates(start_date, end_date, api_key)
movies_details = {}

for movie_id, movie_title in movies_data:
    movie_details, reviews_data = get_movie_details_and_reviews(movie_id, api_key)
    movies_details[movie_id] = {
        'title': movie_title,
        'details': movie_details,
        'reviews': reviews_data
    }

# Ne lancer qu'une seule fois pour éviter d'écraser le fichier JSON

with open('movies_details.json', 'w') as json_file: 
   json.dump(movies_details, json_file, indent=4)

KeyboardInterrupt: 

Maintenant que nous avons récupérées toutes les données utiles à notre projet dans un fichier "movies_details.json", nous allons pouvoir les exploiter.

In [43]:
movies_details

{76600: {'title': 'Avatar: The Way of Water',
  'details': {'adult': False,
   'backdrop_path': '/8rpDcsfLJypbO6vREc0547VKqEv.jpg',
   'belongs_to_collection': {'id': 87096,
    'name': 'Avatar Collection',
    'poster_path': '/gC3tW9a45RGOzzSh6wv91pFnmFr.jpg',
    'backdrop_path': '/gxnvX9kF7RRUQYvB52dMLPgeJkt.jpg'},
   'budget': 460000000,
   'genres': [{'id': 878, 'name': 'Science Fiction'},
    {'id': 12, 'name': 'Adventure'},
    {'id': 28, 'name': 'Action'}],
   'homepage': 'https://www.avatar.com/movies/avatar-the-way-of-water',
   'id': 76600,
   'imdb_id': 'tt1630029',
   'original_language': 'en',
   'original_title': 'Avatar: The Way of Water',
   'overview': 'Set more than a decade after the events of the first film, learn the story of the Sully family (Jake, Neytiri, and their kids), the trouble that follows them, the lengths they go to keep each other safe, the battles they fight to stay alive, and the tragedies they endure.',
   'popularity': 292.293,
   'poster_path': '

Avant de se lancer dans l'analyse de sentiments, nous allons d'abord faire des vérifications sur les données inscritent dans notre fichier json. Nous allons requêter ces données pour voir si elles sont bien présentes et si elles sont bien au format attendu.

On peut par exemple demander le nombre de films dans notre json :

In [44]:
with open('movies_details.json', 'r') as json_file:
    movies_details = json.load(json_file)

num_films = len(movies_details)

print("Nombres de  films:", num_films)

Nombres de  films: 20


On peut également demander le nombre d'avis pour chaque film :

In [18]:
import json

try:
    with open('movies_details.json', 'r') as file:
        movies_details = json.load(file)
    reviews_count = {}
    for movie in movies_details.values():
        num_reviews = len(movie['reviews']['results'])
        movie_id = movie['reviews']['id']
        reviews_count[movie_id] = num_reviews
    for movie_id, count in reviews_count.items():
        print(f"Film ID {movie_id}: Nombre d'avis = {count}")
except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")


Film ID 646385: Nombre d'avis = 9
Film ID 774825: Nombre d'avis = 1
Film ID 557946: Nombre d'avis = 2
Film ID 899082: Nombre d'avis = 0
Film ID 522016: Nombre d'avis = 6
Film ID 575322: Nombre d'avis = 0
Film ID 245842: Nombre d'avis = 3
Film ID 803114: Nombre d'avis = 5
Film ID 926899: Nombre d'avis = 0
Film ID 916740: Nombre d'avis = 1
Film ID 854867: Nombre d'avis = 1
Film ID 825808: Nombre d'avis = 0
Film ID 591120: Nombre d'avis = 1
Film ID 844398: Nombre d'avis = 1
Film ID 760336: Nombre d'avis = 2
Film ID 698508: Nombre d'avis = 1
Film ID 760926: Nombre d'avis = 1
Film ID 778810: Nombre d'avis = 0
Film ID 661791: Nombre d'avis = 0
Film ID 800425: Nombre d'avis = 0


Maintenant on test d'afficher les avis sur un film en particulier : je prends par exemple l'ID du film x : "646385" et je récupère les avis sur ce film :

In [23]:
fichier_json = 'movies_details.json'
movie_id_to_check = 646385

try:
    with open(fichier_json, 'r') as file:
        movies_details = json.load(file)

    selected_movie = movies_details[str(movie_id_to_check)]
    reviews = selected_movie['reviews']['results']
    
    for review in reviews:
        print(f"Auteur : {review['author']}")
        print(f"Rating : {review['author_details']['rating']}")
        print(f"Contenu : {review['content']}\n")

except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

Auteur : garethmb
Rating : None
Contenu : It is hard to believe it has been 25 years since “Scream” premiered and in doing so; brought new life into the slasher genre which had become stale due to an abundance of direct to DVD and cable releases combined with many uninspired and stale theatrical releases.

The movie was a massive success and launched a very successful trilogy and made the Ghostface killer and many moments from the series Pop Culture mainstays.

After a hiatus; “Scream 4” arrived and continued the series and in 2022; “Scream” has arrived which continues the story to the eager anticipation of fans.

When a young girl named Tara (Jenna Ortega); is brutally attacked in a fashion very similar to the Ghostface Killer; her sister Sam (Melissa Barrera); returns to Woodsboro to be by her side while Tara recuperates.

Sam is joined by her boyfriend Richie (Jack Quaid), who has no idea about the history of Woodsboro nor the popular “Stab” film series that is based on the events o

## Idées pour la suite du projet :

- Créer l'application interactive en python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques.
- Analyser les mots clés les plus utilisés dans les commentaires récupérés sur TMDB.
- Faire une analyse de sentiments sur les commentaires récupérés sur TMDB.
- Faire une moyenne de notes de sentiments pour chaque film.
- Comparer les notes de sentiments des avis avec les notes de films attribuées par les utilisateurs.


#### <u> Création de l'application interactive</u>

In [36]:
from flask import Flask, render_template, request
import json

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    with open('movies_details.json', 'r') as file:
        movies_details = json.load(file)
    
    movies_list = [(movie['reviews']['id'], movie['title']) 
                for movie in movies_details.values() 
                if 'reviews' in movie and 'id' in movie['reviews'] and 'title' in movie]

    return render_template('index.html', movies_list=movies_list)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Dec/2023 17:22:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2023 17:22:59] "POST / HTTP/1.1" 200 -


#### <u> Analyse des mots clés les plus utilisés dans les commentaires récupérés sur TMDB</u>

In [35]:
import json
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    words = [word for word in text.split() if word not in stop_words]
    return words

def analyze_word_frequency(movie_reviews):
    all_words = []
    for review in movie_reviews:
        all_words.extend(clean_text(review['content']))
    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(10)
    return most_common_words

with open('movies_details.json', 'r') as file:
    movies_details = json.load(file)

word_frequencies_per_movie = {}
for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    most_common_words = analyze_word_frequency(reviews)
    word_frequencies_per_movie[movie_id] = most_common_words

for movie_id, frequencies in word_frequencies_per_movie.items():
    print(f"Film ID {movie_id}: Mots les plus fréquents - {frequencies}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...


Film ID 646385: Mots les plus fréquents - [('scream', 27), ('like', 13), ('new', 11), ('film', 11), ('movie', 10), ('one', 9), ('horror', 8), ('series', 7), ('first', 7), ('ghostface', 6)]
Film ID 774825: Mots les plus fréquents - [('ice', 3), ('age', 3), ('buck', 3), ('would', 2), ('disney', 2), ('film', 2), ('animation', 2), ('characters', 2), ('bumbling', 2), ('ineptitude', 2)]
Film ID 557946: Mots les plus fréquents - [('last', 3), ('looks', 3), ('film', 3), ('mystery', 3), ('movie', 2), ('bad', 2), ('book', 2), ('way', 2), ('bland', 2), ('every', 2)]
Film ID 899082: Mots les plus fréquents - []
Film ID 522016: Mots les plus fréquents - [('cast', 5), ('chastain', 5), ('film', 5), ('action', 5), ('movie', 5), ('potential', 4), ('great', 3), ('jessica', 3), ('lupita', 3), ('nyong', 3)]
Film ID 575322: Mots les plus fréquents - []
Film ID 245842: Mots les plus fréquents - [('brosnan', 4), ('positive', 4), ('daughter', 3), ('story', 3), ('good', 3), ('way', 3), ('pierce', 3), ('kaya', 

[nltk_data]   Unzipping corpora\stopwords.zip.


Après réfléxion, au vu du peu d'avis récupérés sur TMDB voir même de l'absence d'avis pour certains films, nous avons décidé d'élargir notre base de données en récupérant les données des films sur une plus large période. Nous avons donc décidé de récupérer les données des films sortis pendant l'année 2022 en retirant les films qui n'avaient pas d'avis sur TMDB.

#### <u> Analyse de sentiments sur les commentaires récupérés sur TMDB</u>